In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [3]:
Data=pd.read_csv('drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/malyalam/malayalam_hope_train.csv',names=["text","label","NaN"], sep='\t')
Data.pop('NaN')
Data.head(9)

,text,label
0,@arya s nair may be athile karthikayude charctr bisexual ayirikkum,Non_hope_speech
1,വാങ്ങിയത് എന്ത് കുന്തം ആയാലും കളയാൻ പറ്റില്ലല്ലോ ഇനി വാങ്ങാതിരിക്കാൻ ശ്രദ്ധിക്കുക Jai India,Non_hope_speech
2,മാറുമറയ്ക്കാൻ നടത്തിയ സമരം ഒരു previlege issue ആയിരുന്നു...അത് ഈ വിഷയവുമായി കമ്പയർ ചെയ്യാൻ പറ്റ...,Hope_speech
3,ഇഷാനെ കൂടുതല് കെട്ടിപിടിക്കേണ്ട ഞങ്ങൾക്കറിയാം ഇഷാന് സൂര്യയോട് ഇച്ചിരി കൊഞ്ചൽ കൂടുതലാണെന്ന്,Non_hope_speech
4,ഇന്ത്യ ഇസ്‌ലാമിക രാജ്യമല്ല.. ഇന്ത്യൻ ഭരണഘടന മാത്രമേ ഇന്ത്യൻ പൗരന്മാർ നോക്കേണ്ടതുളൂ. ഇന്ത്യൻ നിയ...,Hope_speech
5,Xenderin pakaram okke entha ubayokhikkuka,Non_hope_speech
6,ചൈനയുടെ കാര്യം അവര് നോക്കട്ടെ.. nഇന്ത്യയിൽ പെട്രോളിന് ദിവസവും വിലക്കുട്ടി കൊണ്ടിരിക്കുകയാണ് അതിന...,Non_hope_speech
7,@Akhil Dev സോറി ഇപ്പഴാ മനസിലായത്,Non_hope_speech
8,നിങ്ങൾക് നല്ലത് വരും.....,Hope_speech


In [4]:
data=Data[['text','label']]
#data.head()

In [5]:
import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)
Data['text']=Data['text'].apply(lambda x: deEmojify(x))

In [6]:
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
Data['text']=Data['text'].apply(lambda x:remove_URL(x))
Data['text']=Data['text'].str.replace(r'\d+','')           #remove numbers


In [7]:
## task into array
## task into array
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(Data['label'])
print (labelEncode.classes_)
label=labelEncode.transform(Data['label'])
label

['Hope_speech' 'Non_hope_speech' 'not-malayalam']


array([1, 1, 0, ..., 1, 0, 0])

# tokenizer


In [8]:
#lemmatizer and tokenization   (good for noun)
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
Punctuations= "!'$%&()*+,-./:;”<=>?[\\]^_`{|}~\t\n"

wordnet_lemmatizer=WordNetLemmatizer()   #lemmatizer object
tok =nltk.tokenize.WhitespaceTokenizer() #Tokenizer object
def lemma(sentence):
    token_words=tok.tokenize(sentence)    #Tokenization
    token_words
    for word in token_words:
        if word in Punctuations:
            token_words.remove(word)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(wordnet_lemmatizer.lemmatize(word))      #Stemming
        stem_sentence.append(" ")                      #Join words
    return "".join(stem_sentence)

Data['Lemma']=Data['text'].apply(lambda x:lemma(x))
Data[['Lemma']].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,Lemma
0,@arya s nair may be athile karthikayude charctr bisexual ayirikkum
1,വാങ്ങിയത് എന്ത് കുന്തം ആയാലും കളയാൻ പറ്റില്ലല്ലോ ഇനി വാങ്ങാതിരിക്കാൻ ശ്രദ്ധിക്കുക Jai India
2,മാറുമറയ്ക്കാൻ നടത്തിയ സമരം ഒരു previlege issue ആയിരുന്നു...അത് ഈ വിഷയവുമായി കമ്പയർ ചെയ്യാൻ പറ്റി...
3,ഇഷാനെ കൂടുതല് കെട്ടിപിടിക്കേണ്ട ഞങ്ങൾക്കറിയാം ഇഷാന് സൂര്യയോട് ഇച്ചിരി കൊഞ്ചൽ കൂടുതലാണെന്ന്
4,ഇന്ത്യ ഇസ്‌ലാമിക രാജ്യമല്ല.. ഇന്ത്യൻ ഭരണഘടന മാത്രമേ ഇന്ത്യൻ പൗരന്മാർ നോക്കേണ്ടതുളൂ. ഇന്ത്യൻ നിയമ...


In [9]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf1 = TfidfVectorizer(ngram_range=(1,1),min_df =2,max_df =0.5)                #tfidf object   for 1 gramm        
features1 = tfidf1.fit_transform(Data['Lemma'])  
vector1=features1.toarray()
vector1.shape

(8564, 5650)

In [10]:
tfidf2 = TfidfVectorizer(ngram_range=(2,2),min_df =2,max_df =0.5)                #tfidf object   for 2 gramm        
features2 = tfidf2.fit_transform(Data['Lemma'])  
vector2=features2.toarray()
vector2.shape

(8564, 7987)

In [11]:
tfidf12 = TfidfVectorizer(ngram_range=(1,2),min_df =2,max_df =0.5)                #tfidf object   for 1,2 gramm        
features12 = tfidf12.fit_transform(Data['Lemma'])  
vector12=features12.toarray()
vector12[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
tfidf123 = TfidfVectorizer(ngram_range=(1,3),min_df =2,max_df =0.5)                #tfidf object   for 123gramm        
features123 = tfidf123.fit_transform(Data['Lemma'])  
vector123=features123.toarray()
vector123.shape

(8564, 17565)

#spliting into training and validation

In [13]:
from sklearn.model_selection import train_test_split
#train_1,val_1,train_2,val_2,train_12,val_12,train_123,val_123,train,test =train_test_split(vector1,vector2,vector12,vector123,label,test_size=0.30,random_state=24)

In [14]:
#print(train_1.shape)
#print(train_2.shape)
#print(train_12.shape)
#print(train_123.shape)
#print(train.shape)

In [15]:
#print(val_1.shape)
#print(val_2.shape)
#print(val_12.shape)
#print(val_123.shape)
#print(test.shape)

#testing DATA

In [16]:
tDATA=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/malyalam/malayalam_hope_dev.csv',names=["text","label","NaN"], sep='\t')
tDATA.pop('NaN')

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1065   NaN
1066   NaN
1067   NaN
1068   NaN
1069   NaN
Name: NaN, Length: 1070, dtype: float64

In [17]:
#tDATA['text']=tDATA['tweet'].apply(lambda x:remove_sw(x))
tDATA['text']=tDATA['text'].apply(lambda x: deEmojify(x))
tDATA['text']=tDATA['text'].str.replace(r'\d+','') 
#tDATA.head()
tDATA['Lemma']=tDATA['text'].apply(lambda x:lemma(x))


In [18]:
t_features1 = tfidf1.transform(tDATA['Lemma'])  
t_vector1=t_features1.toarray()
t_vector1.shape

(1070, 5650)

In [19]:
t_features2 = tfidf2.transform(tDATA['Lemma'])  
t_vector2=t_features2.toarray()
t_vector2.shape

(1070, 7987)

In [20]:
t_features12 = tfidf12.transform(tDATA['Lemma'])  
t_vector12=t_features12.toarray()
t_vector12.shape

(1070, 13637)

In [21]:
t_features123 = tfidf123.transform(tDATA['Lemma'])  
t_vector123=t_features123.toarray()
t_vector123.shape

(1070, 17565)

In [22]:
t_label=labelEncode.transform(tDATA['label'])
t_label


array([0, 1, 0, ..., 1, 1, 1])

# SVM

#for 1 gram

In [23]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.svm import SVC
model_SVM = SVC()
model_SVM.fit(vector1,label)
pred_SVM = model_SVM.predict(t_vector1)
print("SVM")
print("Accuracy score =", accuracy_score(t_label, pred_SVM))
print(metrics.classification_report(t_label,pred_SVM))

SVM
Accuracy score = 0.8102803738317756
              precision    recall  f1-score   support

           0       0.75      0.33      0.45       190
           1       0.82      0.96      0.89       784
           2       0.73      0.51      0.60        96

    accuracy                           0.81      1070
   macro avg       0.77      0.60      0.65      1070
weighted avg       0.80      0.81      0.78      1070



#for 2 gram

In [24]:
model_SVM2 = SVC()
model_SVM2.fit(vector2, label)
pred_SVM = model_SVM2.predict(t_vector2)
print("SVM")
print("Accuracy score =", accuracy_score(t_label, pred_SVM))
print(metrics.classification_report(t_label,pred_SVM))

SVM
Accuracy score = 0.7728971962616823
              precision    recall  f1-score   support

           0       0.69      0.20      0.31       190
           1       0.78      0.98      0.87       784
           2       0.66      0.24      0.35        96

    accuracy                           0.77      1070
   macro avg       0.71      0.47      0.51      1070
weighted avg       0.75      0.77      0.72      1070



#for 12 gram

In [25]:
model_SVM12 = SVC()
model_SVM12.fit(vector12,label)
pred_SVM = model_SVM12.predict(t_vector12)
print("SVM")
print("Accuracy score =", accuracy_score(t_label, pred_SVM))
print(metrics.classification_report(t_label,pred_SVM))

SVM
Accuracy score = 0.8102803738317756
              precision    recall  f1-score   support

           0       0.76      0.33      0.46       190
           1       0.82      0.97      0.89       784
           2       0.74      0.47      0.57        96

    accuracy                           0.81      1070
   macro avg       0.77      0.59      0.64      1070
weighted avg       0.80      0.81      0.78      1070



#for 123 gram

In [26]:
model_SVM123 = SVC()
model_SVM123.fit(vector123,label)
pred_SVM = model_SVM123.predict(t_vector123)
print("SVM")
print("Accuracy score =", accuracy_score(t_label, pred_SVM))
print(metrics.classification_report(t_label,pred_SVM))

SVM
Accuracy score = 0.811214953271028
              precision    recall  f1-score   support

           0       0.77      0.33      0.46       190
           1       0.82      0.97      0.89       784
           2       0.74      0.47      0.57        96

    accuracy                           0.81      1070
   macro avg       0.78      0.59      0.64      1070
weighted avg       0.80      0.81      0.78      1070



#RANDOM FOREST

In [27]:
#for 1 gramm|

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,max_depth=None,min_samples_split=2, random_state=0)
rf.fit(vector1,label)
pred_rf = rf.predict(t_vector1)
print("random")
print("Accuracy score =", accuracy_score(t_label, pred_rf))
print(metrics.classification_report(t_label, pred_rf))

random
Accuracy score = 0.8093457943925234
              precision    recall  f1-score   support

           0       0.70      0.35      0.47       190
           1       0.83      0.95      0.89       784
           2       0.69      0.59      0.64        96

    accuracy                           0.81      1070
   macro avg       0.74      0.63      0.66      1070
weighted avg       0.80      0.81      0.79      1070



In [28]:
# for 2 gram
rf.fit(vector2,label)
pred_rf = rf.predict(t_vector2)
print("random")
print("Accuracy score =", accuracy_score(t_label, pred_rf))
print(metrics.classification_report(t_label, pred_rf))

random
Accuracy score = 0.7813084112149533
              precision    recall  f1-score   support

           0       0.65      0.32      0.42       190
           1       0.80      0.95      0.87       784
           2       0.65      0.31      0.42        96

    accuracy                           0.78      1070
   macro avg       0.70      0.53      0.57      1070
weighted avg       0.76      0.78      0.75      1070



In [29]:
# for 12 gram
rf.fit(vector12,label)
pred_rf12 = rf.predict(t_vector12)
print("random")
print("Accuracy score =", accuracy_score(t_label, pred_rf12))
print(metrics.classification_report(t_label, pred_rf12))

random
Accuracy score = 0.8158878504672897
              precision    recall  f1-score   support

           0       0.74      0.35      0.47       190
           1       0.83      0.96      0.89       784
           2       0.72      0.59      0.65        96

    accuracy                           0.82      1070
   macro avg       0.76      0.63      0.67      1070
weighted avg       0.81      0.82      0.79      1070



In [30]:
#for 123 gram
rf.fit(vector123,label)
pred_rf123 = rf.predict(t_vector123)
print("random")
print("Accuracy score =", accuracy_score(t_label, pred_rf123))
print(metrics.classification_report(t_label, pred_rf123))


random
Accuracy score = 0.8084112149532711
              precision    recall  f1-score   support

           0       0.74      0.31      0.44       190
           1       0.83      0.95      0.89       784
           2       0.67      0.60      0.64        96

    accuracy                           0.81      1070
   macro avg       0.75      0.62      0.65      1070
weighted avg       0.80      0.81      0.78      1070



#LOGESTIC REGRESSION

In [31]:
# FOR 1 gram
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(vector1,label)
pred_LR = LR.predict(t_vector1)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(t_label, pred_LR))
print(metrics.classification_report(t_label, pred_LR ))

Logistic Regression
Accuracy score = 0.8037383177570093
              precision    recall  f1-score   support

           0       0.70      0.36      0.47       190
           1       0.82      0.96      0.88       784
           2       0.70      0.45      0.55        96

    accuracy                           0.80      1070
   macro avg       0.74      0.59      0.63      1070
weighted avg       0.79      0.80      0.78      1070



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [32]:
# for 2 gram

LR.fit(vector2,label)
pred_LR = LR.predict(t_vector2)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(t_label, pred_LR))
print(metrics.classification_report(t_label, pred_LR ))

Logistic Regression
Accuracy score = 0.7897196261682243
              precision    recall  f1-score   support

           0       0.77      0.28      0.42       190
           1       0.79      0.98      0.88       784
           2       0.70      0.24      0.36        96

    accuracy                           0.79      1070
   macro avg       0.75      0.50      0.55      1070
weighted avg       0.78      0.79      0.75      1070



In [33]:
#for 12 gram
LR.fit(vector12,label)
pred_LR12 = LR.predict(t_vector12)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(t_label, pred_LR12))
print(metrics.classification_report(t_label, pred_LR12 ))

Logistic Regression
Accuracy score = 0.8168224299065421
              precision    recall  f1-score   support

           0       0.75      0.40      0.52       190
           1       0.83      0.96      0.89       784
           2       0.75      0.45      0.56        96

    accuracy                           0.82      1070
   macro avg       0.78      0.60      0.66      1070
weighted avg       0.81      0.82      0.80      1070



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [34]:
# for 123 gram
LR.fit(vector123,label)
pred_LR123 = LR.predict(t_vector123)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(t_label, pred_LR123))
print(metrics.classification_report(t_label, pred_LR123 ))

Logistic Regression
Accuracy score = 0.8158878504672897
              precision    recall  f1-score   support

           0       0.75      0.40      0.52       190
           1       0.83      0.96      0.89       784
           2       0.74      0.45      0.56        96

    accuracy                           0.82      1070
   macro avg       0.77      0.60      0.66      1070
weighted avg       0.81      0.82      0.79      1070



#KNeighbors                                        is bad

In [35]:

#for 1 gram

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(vector1,label)
pred_KNN = neigh.predict(t_vector1)
print("KNN")
print("Accuracy score =", accuracy_score(t_label, pred_KNN))
print(metrics.classification_report(t_label,pred_KNN ))

KNN
Accuracy score = 0.7663551401869159
              precision    recall  f1-score   support

           0       0.69      0.11      0.18       190
           1       0.79      0.96      0.87       784
           2       0.52      0.49      0.51        96

    accuracy                           0.77      1070
   macro avg       0.67      0.52      0.52      1070
weighted avg       0.75      0.77      0.71      1070



In [36]:
# for 2 gram
neigh.fit(vector2,label)
pred_KNN = neigh.predict(t_vector2)
print("KNN")
print("Accuracy score =", accuracy_score(t_label, pred_KNN))
print(metrics.classification_report(t_label,pred_KNN ))

KNN
Accuracy score = 0.7570093457943925
              precision    recall  f1-score   support

           0       0.58      0.15      0.24       190
           1       0.77      0.97      0.86       784
           2       0.61      0.23      0.33        96

    accuracy                           0.76      1070
   macro avg       0.66      0.45      0.48      1070
weighted avg       0.72      0.76      0.70      1070



In [37]:
# for 12 gram
neigh.fit(vector12,label)
pred_KNN12 = neigh.predict(t_vector12)
print("KNN")
print("Accuracy score =", accuracy_score(t_label, pred_KNN12))
print(metrics.classification_report(t_label,pred_KNN12 ))

KNN
Accuracy score = 0.7644859813084112
              precision    recall  f1-score   support

           0       0.58      0.12      0.19       190
           1       0.78      0.97      0.87       784
           2       0.58      0.41      0.48        96

    accuracy                           0.76      1070
   macro avg       0.65      0.50      0.51      1070
weighted avg       0.73      0.76      0.71      1070



In [38]:
# for 123 gram
neigh.fit(vector123,label)
pred_KNN123 = neigh.predict(t_vector123)
print("KNN")
print("Accuracy score =", accuracy_score(t_label, pred_KNN123))
print(metrics.classification_report(t_label,pred_KNN123 ))

KNN
Accuracy score = 0.7635514018691589
              precision    recall  f1-score   support

           0       0.62      0.11      0.19       190
           1       0.78      0.97      0.86       784
           2       0.57      0.40      0.47        96

    accuracy                           0.76      1070
   macro avg       0.66      0.49      0.51      1070
weighted avg       0.73      0.76      0.71      1070



# NAIVE BAYES 

In [39]:
# for 1 gram
from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(vector1,label)
pred_naive = naive.predict(t_vector1)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(t_label, pred_naive))
print(metrics.classification_report(t_label, pred_naive ))

Naive Bayes
Accuracy score = 0.5205607476635514
              precision    recall  f1-score   support

           0       0.26      0.70      0.38       190
           1       0.85      0.45      0.59       784
           2       0.48      0.72      0.57        96

    accuracy                           0.52      1070
   macro avg       0.53      0.62      0.52      1070
weighted avg       0.71      0.52      0.55      1070



In [40]:
# for 2 gram

naive.fit(vector2,label)
pred_naive = naive.predict(t_vector2)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(t_label, pred_naive))
print(metrics.classification_report(t_label, pred_naive ))

Naive Bayes
Accuracy score = 0.41869158878504675
              precision    recall  f1-score   support

           0       0.36      0.47      0.41       190
           1       0.85      0.34      0.49       784
           2       0.18      0.95      0.30        96

    accuracy                           0.42      1070
   macro avg       0.47      0.59      0.40      1070
weighted avg       0.71      0.42      0.46      1070



In [41]:
# for 12 gram

naive.fit(vector12,label)
pred_naive = naive.predict(t_vector12)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(t_label, pred_naive))
print(metrics.classification_report(t_label, pred_naive ))

Naive Bayes
Accuracy score = 0.6
              precision    recall  f1-score   support

           0       0.29      0.59      0.39       190
           1       0.85      0.59      0.69       784
           2       0.50      0.72      0.59        96

    accuracy                           0.60      1070
   macro avg       0.55      0.63      0.56      1070
weighted avg       0.72      0.60      0.63      1070



In [42]:
# for 123 gram

naive.fit(vector123,label)
pred_naive = naive.predict(t_vector123)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(t_label, pred_naive))
print(metrics.classification_report(t_label, pred_naive ))

Naive Bayes
Accuracy score = 0.6056074766355141
              precision    recall  f1-score   support

           0       0.29      0.58      0.39       190
           1       0.85      0.60      0.70       784
           2       0.50      0.72      0.59        96

    accuracy                           0.61      1070
   macro avg       0.55      0.63      0.56      1070
weighted avg       0.72      0.61      0.64      1070



#GradientBoosting

In [ ]:
# for 1 gram
from sklearn.ensemble import GradientBoostingClassifier
gradient = GradientBoostingClassifier(n_estimators=100,max_depth=None,min_samples_split=2, random_state=0)
gradient.fit(vector1,label)
pred_gradient = gradient.predict(t_vector1)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(t_label,pred_gradient))
print(metrics.classification_report(t_label, pred_gradient ))

In [ ]:
# for 2 gram
gradient.fit(vector2,label)
pred_gradient = gradient.predict(t_vector2)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(t_label,pred_gradient))
print(metrics.classification_report(t_label, pred_gradient ))

In [ ]:
# for 12 gram
gradient.fit(vector12,label)
pred_gradient12 = gradient.predict(t_vector12)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(t_label,pred_gradient12))
print(metrics.classification_report(t_label, pred_gradient12 ))

In [ ]:
# for 123 gram
gradient.fit(vector123,label)
pred_gradient123 = gradient.predict(t_vector123)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(t_label,pred_gradient123))
print(metrics.classification_report(t_label, pred_gradient123 ))

#DECISION TREE

In [ ]:
# for 1 gram
from sklearn.tree import DecisionTreeClassifier
decision = DecisionTreeClassifier()
decision.fit(vector1,label)
pred_decision = decision.predict(t_vector1)
print("Decision Tree")
print("Accuracy score =", accuracy_score(t_label,pred_decision))
print(metrics.classification_report(t_label,pred_decision ))

In [ ]:
# for 2 gram
decision.fit(vector2,label)
pred_decision2 = decision.predict(t_vector2)
print("Decision Tree")
print("Accuracy score =", accuracy_score(test,pred_decision2))
print(metrics.classification_report(test,pred_decision2 ))

In [ ]:
# for 12 gram
decision.fit(vector12,label)
pred_decision12 = decision.predict(t_vector12)
print("Decision Tree")
print("Accuracy score =", accuracy_score(t_label,pred_decision12))
print(metrics.classification_report(t_label,pred_decision12 ))

In [ ]:
# for 123 gram
decision.fit(vector123,label)
pred_decision123 = decision.predict(t_vector123)
print("Decision Tree")
print("Accuracy score =", accuracy_score(t_label,pred_decision123))
print(metrics.classification_report(t_label,pred_decision123 ))